# Back-Propagation Example
## on a neural network with single hidden layer with 3 units

### Data Pre-processing

In [1]:
import numpy as np
import pandas as pd

In [2]:
file=pd.read_csv("binary.csv")

In [3]:
data=pd.concat([file,pd.get_dummies(file['rank'],prefix="rank")],axis=1)

In [4]:
data=data.drop("rank",axis=1)

In [5]:
for field in ['gre','gpa']:
    mean,stnd=data[field].mean(),data[field].std()
    data.loc[:,field]=(data[field]-mean)/stnd

In [6]:
np.random.seed(28)

### Spliting the dataset in 7:3 ratio 

In [18]:
sample=np.random.choice(data.index,size=int(len(data)*0.7),replace=False)

In [19]:
data_train,data_test=data.iloc[sample],data.drop(sample)

In [20]:
features,targets=data_train.drop('admit',axis=1),data_train['admit']

In [21]:
features_test,targets_test=data_test.drop('admit',axis=1),data_test['admit']

### Defining activation function and hyperparameters

In [22]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [31]:

learnrate=0.004
epochs=1000
hiddenunits=3

In [24]:
nr,nf=features.shape

In [25]:
last_loss=None

In [26]:
weights_to_hidden=np.random.normal(scale=1/nf**0.5,size=(nf,hiddenunits))
weights_to_output=np.random.normal(scale=1/nf**0.5,size=hiddenunits)

# The Algorithm
### -Set delta weights,ie weight steps as 0,both of weights to hidden and output layer
### - For each record in data:
     -Make a forward pass calculating the final output of the network.
     -Then calculate the error gradient at the output unit= (target-output)*derivative of the output function
     -Propagate errors to the hidden layer's all nodes.Say at unit j,
        >Error Gradient at j=Weight_from_j_to_output*Error_Gradient_at_Output*derivative of activation function at node j
     -Update the weights steps:
       >Weights steps for weights to output from j +=Error_Gradient_at_Output * a_at_j  #a is output from hidden layer unit j
        >Weights steps for weights to hidden unit j from i input += Error_Grad_at_j * a_at_i is the input 
### - Update the weights:
       Weights to hidden unit j from i += learnrate * weightstep_at_ij / n_records
       Weights from hidden unit j  to output += learnrate * weightstep_at_j / n_records
## Repeat these steps for e epochs.     

In [32]:
for e in range(epochs):
    del_w_ij=np.zeros(weights_to_hidden.shape)
    del_w_j=np.zeros(weights_to_output.shape)
    for x,y in zip(features.values,targets):
        hidden_output=sigmoid(np.dot(x,weights_to_hidden))
        output=sigmoid(np.dot(hidden_output,weights_to_output))
        output_error=y-output
        output_errgradient=output_error*output*(1-output)
        #Calculating Hidden Layer contribution to the error
        errh=np.dot(output_errgradient,weights_to_output)
        error_termh=errh * hidden_output *(1-hidden_output)
        del_w_ij+=error_termh*x[:,None]
        del_w_j+=output_errgradient*hidden_output
    weights_to_hidden+=(learnrate*del_w_ij)/nr
    weights_to_output+=(learnrate*del_w_j)/nr
    
    if e % (epochs/10)==0:
        houtput=sigmoid(np.dot(x,weights_to_hidden))
        output=sigmoid(np.dot(houtput,weights_to_output))
        error=targets-output
        m2er=np.mean(error**2)
        if last_loss and m2er>last_loss:
            print("Loss Increasing=",m2er)
        else:
            print("Loss:",m2er)
        last_loss=m2er

Loss: 0.222010587237634
Loss: 0.22197000408413436
Loss: 0.22193036714866704
Loss: 0.2218916569637991
Loss: 0.2218538544827864
Loss: 0.22181694106968303
Loss: 0.2217808984896947
Loss: 0.22174570889979323
Loss: 0.22171135483955812
Loss: 0.2216778192222626


## Testing Out the Accuracy of the model

In [33]:
hidout=sigmoid(np.dot(features_test,weights_to_hidden))
outputtest=sigmoid(np.dot(hidout,weights_to_output))
predic=outputtest>0.5
accuracy=np.mean(predic==targets_test)
print(accuracy)

0.7083333333333334
